This notebook is created at GESIS for general public to test Gender Guesser package on gender inference from names. It is a supplement to the GESIS Gender Inference website. You can upload your own list of names. All data is deleted afterwards.

In [ ]:
import gender_guesser.detector as gender
import ipywidgets as widgets
import pandas as pd
import codecs
from io import StringIO
from IPython.display import clear_output

Gender_Guesser will return one of the following results: unknown (name not found), andy (androgynous, probability of male is the same as female), male, female, mostly_male, or mostly_female. 

In [ ]:
name = widgets.Text(
    value='Bob',
    description='Name:',
    disabled=False
)
run = widgets.Button(description="Run")
display(name, run)

def on_run_clicked(b):
    '''Predicting gender from one name'''
    d = gender.Detector()
    print(f"Gender of the given name: {d.get_gender(name.value)}")
    
    
run.on_click(on_run_clicked)

I18N is also fully supported:

In [ ]:
d = gender.Detector()
print(d.get_gender(u"\xc1lfr\xfan"))  # u"Álfrún"

Gender_Guesser also takes country name as a parameter:

In [ ]:
print(d.get_gender(u"Jamie", u'great_britain'))
print(d.get_gender(u"Jamie", u'germany'))

Upload your csv file using the button below to run Gender_Guesser on a list of names. Each row should have a name (by default detector is case sensitive, you can change it by adding a parameter *case_sensitive=False*) and optionally a country. <br/>
Please make sure the csv file has "," as a separator and no space between the columns

In [ ]:
upload = widgets.FileUpload(
        accept='.csv', 
        multiple=False  
    )
run = widgets.Button(description="Run")
display(upload, run)
global submitted 
submitted = False

def on_run_clicked(b):
    '''Predicting gender from a list of names'''
    
    global submitted
    d = gender.Detector()
    
    #extracting the content of csv file as a string
    uploaded_filename = next(iter(upload.value))
    content = codecs.decode(upload.value[uploaded_filename]['content'], encoding="utf-8")
    
    #converting a string to a dataframe
    datatbl = StringIO(content)
    names_df = pd.read_table(datatbl,sep=',',header=None, index_col=None)
    
    #case with countries
    if len(names_df.columns)>1:
        names_df = names_df.rename(columns={0:"Name", 1:"Country"})
        names_df["Gender"] = names_df.apply(lambda x: d.get_gender(x.Name, x.Country), axis=1)
    #case with only names
    else:
        names_df = names_df.rename(columns={0:"Name"})
        names_df["Gender"] = names_df.apply(lambda x: d.get_gender(x.Name), axis=1)
    if submitted == True:
        clear_output()
        display(upload,run)
    display(names_df)
    submitted = True
    
    
run.on_click(on_run_clicked)